In [ ]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--conf spark.sql.catalogImplementation=in-memory pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [ ]:
data = spark.read.parquet("/user/pavel.klemenkov/lectures/lecture05/credit_fraud/")

In [3]:
data.printSchema()

root
 |-- time: integer (nullable = true)
 |-- amountRange: integer (nullable = true)
 |-- label: integer (nullable = true)
 |-- pcaVector: vector (nullable = true)



In [4]:
data.select("amountRange").show(truncate=False)

+-----------+
|amountRange|
+-----------+
|2          |
|6          |
|7          |
|3          |
|7          |
|7          |
|6          |
|1          |
|4          |
|3          |
|5          |
|7          |
|3          |
|2          |
|4          |
|5          |
|6          |
|7          |
|6          |
|6          |
+-----------+
only showing top 20 rows



We'll be using 3 columns of this dataset:


- `pcaVector`: The PCA transformation of raw transaction data. For this example we'll assume that this PCA transformation occurs as part of some data pipeline before the data reaches us.
- `amountRange`: A value between 0 and 7. The approximate amount of a transaction. The values correspond to 0-1, 1-5, 5-10, 10-20, 20-50, 50-100, 100-200, and 200+ in dollars.
- `label`: 0 or 1. Indicates whether a transaction was fraudulent.

We want to build a model that will predict the label using the `pcaVector` and `amountRange` data. We'll do this by using a pipeline with 3 stages.


1. A `OneHotEncoderEstimator` to build a vector from the `amountRange` column. 
2. A `Vector` assembler to merge our `pcaVector` and `amountRange` vector into our `features` vector. 
3. A `GBTClassifier` to server as our `Estimator`.

Let's start by creating the objects that represent these stages.

In [5]:
from pyspark.ml.feature import OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.classification import GBTClassifier

In [6]:
one_hot = OneHotEncoderEstimator(inputCols=["amountRange"], outputCols=["amountVect"])

In [7]:
assembler = VectorAssembler(inputCols=["amountVect", "pcaVector"], outputCol="features")

In [8]:
estimator = GBTClassifier()

In [9]:
from pyspark.ml.feature import VectorSizeHint

In [10]:
hint = VectorSizeHint(inputCol="pcaVector", size=28)

In [11]:
train, test = data.randomSplit([0.8, 0.2])

In [12]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[
  one_hot,
  hint,
  assembler,
  estimator
])

In [13]:
pipeline_model = pipeline.fit(train)

In [14]:
pipeline_model

PipelineModel_4e2e9f913937dd916519

In [15]:
testDataPath = "/user/pavel.klemenkov/lectures/lecture05/credit-card-fraud-test-data"
test.repartition(20).write.mode("overwrite").parquet(testDataPath)

In [16]:
from pyspark.sql.types import *
from pyspark.ml.linalg import VectorUDT

In [20]:
schema = StructType(fields=[
StructField("time", IntegerType()),
StructField("amountRange", IntegerType()),
StructField("label", IntegerType()),
StructField("pcaVector", VectorUDT())
])

In [21]:
streamingData = spark.readStream\
                     .schema(schema)\
                     .option("maxFilesPerTrigger", 1)\
                     .parquet(testDataPath)

In [22]:
streamingData.isStreaming

True

In [23]:
streamingRates = pipeline_model.transform(streamingData)

In [24]:
streamingRates

DataFrame[time: int, amountRange: int, label: int, pcaVector: vector, amountVect: vector, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [25]:
streaming_query = streamingRates.writeStream\
                                .format("memory")\
                                .outputMode("append")\
                                .queryName("prediction")\
                                .start()

In [26]:
import time
from IPython.display import clear_output

In [27]:
while True:
    clear_output()
    spark.sql("select sum(prediction) from prediction").show()
    time.sleep(5)

+---------------+
|sum(prediction)|
+---------------+
|           97.0|
+---------------+



KeyboardInterrupt: 

In [28]:
streaming_query.isActive

True

In [29]:
streaming_query.stop()

In [30]:
spark.stop()